# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit b5cb66a492b3d970e464e4961b68302a9e38a4fb
Author: Najwa Ez Zine <nezzine@frennes>
Date:   Fri Jun 8 13:54:51 2018 +0200

    Journal update


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl


In [125]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread
import math
import collections
from collections import deque

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 4
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [11]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

A   job is already running, using it. jobid is 1015577


On les range

In [12]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-1.rennes.grid5000.fr'),
 Host('parasilo-10.rennes.grid5000.fr'),
 Host('parasilo-19.rennes.grid5000.fr'),
 Host('parasilo-24.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [13]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [14]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

rennes: Deployment #D-c705e8af-2a86-4589-a9bb-11f69e866bf0 started
rennes: Grab the key file /home/nezzine/.ssh/authorized_keys
rennes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
rennes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
rennes: Launching a deployment on parasilo-[1,10,19,24].rennes.grid5000.fr
rennes: Performing a Deploy[SetDeploymentEnvUntrusted] step
rennes:   switch_pxe
rennes:   reboot
rennes:    * Performing a soft reboot on parasilo-[1,10,19,24].rennes.grid5000.fr
rennes:   wait_reboot
rennes:   send_key_in_deploy_env
rennes:   create_partition_table
rennes:   format_deploy_part
rennes:   mount_deploy_part
rennes:   format_tmp_part
rennes:   format_swap_part
rennes: End of step Deploy[SetDeploymentEnvUntrusted] after 215s
rennes: Performing a Deploy[BroadcastEnvKascade] step
rennes:   send_environment
rennes:    * Broadcast time: 73s
rennes:   manage_admin_post_install
rennes:   m

rennes: Warning: Permanently added 'rennes.grid5000.fr,172.16.111.106' (ECDSA) to the list of known hosts.
rennes: Connection to rennes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [ ]:
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [15]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [16]:
Remote_install.run().ok

True

#### Version Simgrid

In [ ]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


In [17]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [18]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

In [19]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [20]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        line = l[i].split()
        S[int(line[1])] = int(float(line[5]))
    return S

In [21]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return D

In [ ]:
# Example
delayer(0,10,'2018-06-04_12:38')

##### Shortcut : séquentiel

In [22]:
def setup() :
    Src_dir = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
    Src_get = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
    Rslt_dir = \
        execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
    Src_dir.run().ok
    Src_get.run().ok
    Rslt_dir.run().ok


In [ ]:

def parl_exec () :
    setup()
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Parallel simulation

In [222]:
# Thread pour la simulation // opt.
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""
    
    # Paramètrage du thread
    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
    
    # Set la valeur du délai
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    def get_MIN(self):
        return self.MIN
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        
        # make run sur toutes la machine dont le thread s'occupe avec ses paramètres
        execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [24]:
# Test : trajectoire jusqu'à la ième machine raccrochée à la trajectoire initiale
def coherence(tab,i):
     return all(x == True for x in tab[0:i])

In [25]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def format_cmd(data):
    return (' '.join(str(e) for e in data))

In [234]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def unformat_cmd(data):
    return data.split()

In [259]:
# SOURCE : https://stackoverflow.com/questions/3229419/how-to-pretty-print-nested-dictionaries
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [306]:
# SIMULATION
### Simulation simple en diminuant le nbr de machines 
def simulation(N,NPROCS,mode):
    setup() 

    # Dates
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    date = date_precise.split('_')[0]
    
    # Tableau de threads
    machines = init_tab(nodecount,None)
      
    # position trajectoire cohérente courante
    t_curr = 0
    
    # D de la trajectorie cohérente courante
    D_curr = init_tab(NPROCS, 0)
    
    # taille paquet d'itérations
    fragment_size = 25
    
    # Nombre de rounds maximum
    
    fragment_number = int((math.ceil(N/fragment_size)))
    rounds_max = int((math.ceil(fragment_number/nodecount)))
    print(rounds_max)
    
    # zone d'itérations à explorer
    To_explore = collections.deque()
    
    ### Initialisation
    for i in range(fragment_number):
        To_explore.append((i * fragment_size, init_tab(NPROCS, 0)))
    
    curr_round = 0
    sim = {}
    while(True):
        curr_round = curr_round + 1
        
        if(t_curr == fragment_number) :
            break;
            
        """
       if(curr_round > rounds_max):
            break
       """ 
        

        print("Round :  " + str(curr_round) + "   ---------------------------------------------------------\n")
                
        size = len(To_explore)
        print(size)
        for q in range(min(nodecount,size)):
            parameters = To_explore.popleft()
            MIN = parameters[0]
            MAX = MIN + fragment_size
            delay = format_cmd(parameters[1])
            machines[q] = ParSimer(q, MIN, MAX, N, delay)
            print("Machine " + str(q) + " MIN :" + str(MIN) + " MAX : " + str(MAX) + " D : "+ delay)   
            
        # Lancement de la simulation
        for i in range(min(nodecount,size)):
            machines[i].start()
            
        # Fin de simulation
        for i in range(min(nodecount,size)) :
            machines[i].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok       
        for q in range(min(nodecount,size)):
            t = machines[q].get_MIN()
            D = machines[q].get_delay()
            Dprime = format_cmd(delayer(q,NPROCS,date_precise))
            sim.update({t:{D:(Dprime,curr_round,q)}})
            MAX = t + fragment_size
            To_explore.append((MAX,Dprime))
            
        print("Tcurr : "+ str(t_curr)+ " D_curr : "+ str(D_curr)+ " Dprme" + str(Dprime))
        print(sim[t_curr][format_cmd(D_curr)])
        print(sim[t_curr+fragment_size])
        if(sim[t_curr][format_cmd(D_curr)] in sim[t_curr+fragment_size]):
            t_curr = t_curr + fragment_size
            D_curr = sim[t_curr + fragment_size][D_curr][0]
            print("Tcurr : "+ str(t_curr)+ " D_curr : "+ str(D_curr))
 
        #pretty(sim)

In [307]:
# Nbr pcs
NPROCS = 3
    
# Nbr itérations
N = 100

# mode de génération de l'état initial à chaque round
mode = ["FIX","RAND","RAND_INF","RAND_SUP"]

# Lancement de la simulation
simulation(N,NPROCS,mode[1])

1
Round :  1   ---------------------------------------------------------

4
Machine 0 MIN :0 MAX : 25 D : 0 0 0
Machine 1 MIN :25 MAX : 50 D : 0 0 0
Machine 2 MIN :50 MAX : 75 D : 0 0 0
Machine 3 MIN :75 MAX : 100 D : 0 0 0
parasilo-1 => 0 NPROCS = 3 MIN = 0 MAX = 25 N = 100 DELAY = '0 0 0'

parasilo-10 => 1 NPROCS = 3 MIN = 25 MAX = 50 N = 100 DELAY = '0 0 0'

parasilo-19 => 2 NPROCS = 3 MIN = 50 MAX = 75 N = 100 DELAY = '0 0 0'

parasilo-24 => 3 NPROCS = 3 MIN = 75 MAX = 100 N = 100 DELAY = '0 0 0'

Tcurr : 0 D_curr : [0, 0, 0] Dprme1 3 0
('1 5 0', 1, 0)
{'0 0 0': ('1 3 0', 1, 1)}
Round :  2   ---------------------------------------------------------

4
Machine 0 MIN :25 MAX : 50 D : 1   5   0
Machine 1 MIN :50 MAX : 75 D : 1   3   0
Machine 2 MIN :75 MAX : 100 D : 1   3   0
Machine 3 MIN :100 MAX : 125 D : 1   3   0
parasilo-1 => 0 NPROCS = 3 MIN = 25 MAX = 50 N = 100 DELAY = '1   5   0'

parasilo-10 => 1 NPROCS = 3 MIN = 50 MAX = 75 N = 100 DELAY = '1   3   0'

parasilo-19 => 2 NPR

ValueError: could not convert string to float: 'smpitmp-appqAhCKn'

In [288]:
N = 100
fragment_size = 10
t = 1
z=1
D = init_tab((int) (N/fragment_size) ,0)
sim = {t: {z:('First','Second')}, 'eeee':('First','Second') }

print(sim[t][z])
sim[t].update({z:('First','Second')})
print(sim[t])
if 3 in sim[t]:
    print ("yes: " + str(sim[t][2]))
#sim = {t: {D: delayer(i-1,Round,get_host_shortnames[nodes[i]])}, 'eeee': 'First'}


('First', 'Second')
{1: ('First', 'Second')}


#### Publication des résultats

In [ ]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"' Path'

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)